# Building Your First RAG ChatBot with LangChain
## Part 5a: From Documents to Intelligent Q&A in 30 Minutes

**Welcome to the finale of our LangChain RAG series!** 🎉

In the previous articles, we learned:
- **Part 1**: [Document Loading](link) - Getting data into your system
- **Part 2**: [Document Splitting](blog_document_splitting.md) - Optimal chunking strategies
- **Part 3**: [Vector Embeddings](blog_vector_embeddings.md) - Semantic understanding
- **Part 4**: [Vector Databases](link) - Storage and retrieval at scale

**Now we bring it ALL together!** 🚀

In this notebook, you'll build a working RAG chatbot that:
- ✅ Answers questions about YOUR documents
- ✅ Shows sources for answers
- ✅ Runs completely locally (free!)
- ✅ Takes ~30 minutes to set up

---

## What is RAG Again?

**RAG (Retrieval Augmented Generation)** = Smart Q&A over your documents

**The Problem:**
- LLMs don't know about YOUR specific documents
- They can't access real-time or proprietary information
- They "hallucinate" when they don't know answers

**The Solution (RAG):**
1. User asks a question
2. System retrieves relevant document chunks
3. LLM generates answer using those chunks
4. Result: Accurate, grounded responses with sources!

---

**💡 By the end:** You'll have a chatbot answering questions about any PDF, website, or text you provide!

## Architecture Overview

```
┌─────────────┐
│  Documents  │ (PDFs, websites, text)
└──────┬──────┘
       │
       ▼
┌─────────────┐
│   Loader    │ (Read documents)
└──────┬──────┘
       │
       ▼
┌─────────────┐
│  Splitter   │ (Chunk into pieces)
└──────┬──────┘
       │
       ▼
┌─────────────┐
│ Embeddings  │ (Convert to vectors)
└──────┬──────┘
       │
       ▼
┌─────────────┐
│ Vector DB   │ (Store in Chroma)
└──────┬──────┘
       │
┌──────┴──────┐
│             │
▼             ▼
User ──► Retriever ──► LLM ──► Answer
Question     (Find       (Generate
             relevant)   response)
```

**Simple, right?** Let's build it step by step!

## Setup: Install Required Packages

In [2]:
# Install all required packages
!pip install -q langchain langchain-community langchain-core langchain-chroma
!pip install -q langchain-huggingface sentence-transformers
!pip install -q langchain-openai langchain-google-genai google-generativeai langchain-groq langchain-huggingface langchain-tavily
!pip install -q pypdf pymupdf  # For PDF loading
!pip install -q chromadb faiss-cpu # Vector database

print("✓ All packages installed successfully!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 81.7 MB/s eta 0:00:0

## Optional: API Keys Setup

**Two options for LLM:**
1. **OpenAI** (paid, best quality) - Requires API key
2. **Local LLM** (free, good enough) - No API key needed

We'll show both approaches!

In [4]:
#For Google Colab

import os
from google.colab import userdata


OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
GEMINI_API_KEY=userdata.get('GEMINI_API_KEY')
GROQ_API_KEY=userdata.get('GROQ_API_KEY')

---

# Step 1: Load Documents 📄

First, we need documents to answer questions about. We'll use multiple sources.

## Option A: Load from Sample Text (Quick Start)

In [5]:
from langchain_core.documents import Document

# Sample documents about LangChain and RAG
sample_docs = [
    Document(
        page_content="""LangChain is a framework for developing applications powered by language models.
        It enables applications that are context-aware and can reason about information. LangChain provides
        tools for document loading, text splitting, embeddings, vector stores, and chains for building
        complex LLM applications.""",
        metadata={"source": "langchain_intro.txt", "topic": "LangChain Overview"}
    ),
    Document(
        page_content="""Retrieval Augmented Generation (RAG) is a technique that combines information retrieval
        with text generation. It works by first retrieving relevant documents from a knowledge base, then
        passing those documents to a language model to generate accurate, grounded responses. This prevents
        hallucinations and allows LLMs to answer questions about specific, up-to-date information.""",
        metadata={"source": "rag_explanation.txt", "topic": "RAG"}
    ),
    Document(
        page_content="""Vector embeddings are numerical representations of text that capture semantic meaning.
        Similar texts produce similar embedding vectors. Common embedding models include OpenAI's text-embedding-3-small,
        sentence-transformers from HuggingFace like all-MiniLM-L6-v2, and Google's text-embedding-004.
        Embeddings typically have 384 to 1536 dimensions.""",
        metadata={"source": "embeddings_guide.txt", "topic": "Embeddings"}
    ),
    Document(
        page_content="""Chroma is an open-source embedding database designed for AI applications.
        It provides fast similarity search, easy integration with LangChain, and persistent storage.
        Chroma supports metadata filtering and is perfect for building RAG applications. It can run
        locally or in client-server mode.""",
        metadata={"source": "chroma_db.txt", "topic": "Vector Databases"}
    ),
    Document(
        page_content="""Document splitting is crucial for RAG systems. The RecursiveCharacterTextSplitter
        is recommended as it intelligently splits on paragraphs, sentences, and words in order.
        Optimal chunk size is typically 500-1000 characters with 10-20% overlap. The overlap ensures
        context isn't lost at chunk boundaries.""",
        metadata={"source": "splitting_guide.txt", "topic": "Document Splitting"}
    ),
]

print(f"✓ Loaded {len(sample_docs)} sample documents")
print(f"\nSample content: {sample_docs[0].page_content[:100]}...")

✓ Loaded 5 sample documents

Sample content: LangChain is a framework for developing applications powered by language models. 
        It enables...


## Option B: Load from PDF (Your Own Documents)

In [29]:
# Uncomment to load your own PDF
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("/content/Mastering AI Agents.pdf") #upload any PDF for chat
sample_docs = loader.load()
print(f"✓ Loaded {len(sample_docs)} pages from PDF")
print(f"\nSample content: {sample_docs[5].page_content[:100]}...")

#print("💡 To use your own PDF: uncomment above and provide PDF path")

✓ Loaded 93 pages from PDF

Sample content: 6
Mastering AI Agents
Development Issues
LLM Issues
Production Issues
81
82
86
Chapter 5:
Why Most A...


## Option C: Load from Website

In [ ]:
# Uncomment to load from a website
# from langchain_community.document_loaders import WebBaseLoader

# loader = WebBaseLoader("https://python.langchain.com/docs/get_started/introduction")
# sample_docs = loader.load()
# print(f"✓ Loaded content from website")

print("💡 To load from web: uncomment above and provide URL")

---

# Step 2: Split Documents into Chunks 🔪

We split documents into smaller chunks for better retrieval precision.

In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Create text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,        # Size of each chunk
    chunk_overlap=50,      # Overlap between chunks (preserves context)
    length_function=len,   # How to measure chunk size
    separators=["\n\n", "\n", " ", ""]  # Split on paragraphs, then sentences, then words
)

# Split the documents
splits = text_splitter.split_documents(sample_docs)

print(f"✓ Split {len(sample_docs)} documents into {len(splits)} chunks")
print(f"\nSample chunk:")
print(f"Content: {splits[0].page_content}")
print(f"Metadata: {splits[0].metadata}")
print(f"Chunk size: {len(splits[0].page_content)} characters")

✓ Split 93 documents into 288 chunks

Sample chunk:
Content: 2
Mastering AI Agents
Preface
In our previous e-book, “Mastering RAG,” our goal was clear: building enterprise-grade 
RAG systems, productionizing them, monitoring their performance, and improving them. 
At the core of it, we understood how RAG systems enhance an LLM’s ability to work with 
specific knowledge by providing relevant context. 
In this e-book, we’re taking a step further and asking, “How do we use LLMs to
Metadata: {'producer': '', 'creator': '', 'creationdate': '', 'source': '/content/Mastering AI Agents.pdf', 'file_path': '/content/Mastering AI Agents.pdf', 'total_pages': 93, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 1}
Chunk size: 421 characters


**💡 Why splitting matters:**
- Too large: Retrieves irrelevant info, wastes LLM context
- Too small: Loses context, poor answers
- Just right: Precise retrieval, better answers!

---

# Step 3: Create Embeddings 🧮

Convert text chunks into vector embeddings for semantic search.

In [30]:
from langchain_huggingface import HuggingFaceEmbeddings

# Initialize embeddings model (free, runs locally)
embeddings = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2",  # Fast, 384 dimensions
    #model_name="all-mpnet-base-v2",
    model_kwargs={'device': 'cpu'}  # Use 'cuda' if you have GPU
)

# Test the embeddings
test_text = "What is LangChain?"
test_embedding = embeddings.embed_query(test_text)

print(f"✓ Embeddings model loaded")
print(f"Model: all-MiniLM-L6-v2")

print(f"Embedding dimensions: {len(test_embedding)}")
print(f"Sample values: {test_embedding[:5]}")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✓ Embeddings model loaded
Model: all-MiniLM-L6-v2
Embedding dimensions: 384
Sample values: [-0.040557801723480225, 0.013319497928023338, 0.019409090280532837, 0.004821693059056997, -0.061190806329250336]


**Alternative: Use OpenAI Embeddings (if you have API key)**

In [11]:
# Uncomment if using OpenAI
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small",api_key=OPENAI_API_KEY)
print("✓ Using OpenAI embeddings")

print("💡 OpenAI embeddings are higher quality but cost ~$0.02 per 1M tokens")

✓ Using OpenAI embeddings
💡 OpenAI embeddings are higher quality but cost ~$0.02 per 1M tokens


## OPtion3: Use Google Gemini Embedding Model(Free)

In [ ]:
##Use Gemini Embedding model
from langchain_google_genai import GoogleGenerativeAIEmbeddings
gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
Test_query="What is the capital of France?"
Gemini_Embeddings=gemini_embeddings.embed_query(Test_query)
print(Gemini_Embeddings)
print(len(Gemini_Embeddings))
print("✓ Using Google Gemini Embeddings")

print("💡 Google Gemini embeddings are free but cost ~$0.02 per 1M tokens")


[-0.023838477209210396, -0.008524507284164429, 0.010140136815607548, -0.03635908290743828, 0.005881804041564465, 0.017632408067584038, 0.03440266102552414, -0.022488268092274666, -0.037677180022001266, 0.09012677520513535, -0.041247013956308365, -0.07383030652999878, -0.03170005604624748, 0.03297711908817291, -0.027617353945970535, -0.07830660790205002, -0.010694045573472977, 0.01068727858364582, -0.07397930324077606, 0.0229897852987051, 0.0039268983528018, -0.009637449868023396, 0.025337379425764084, -0.008505502715706825, 0.04152005538344383, -0.045486852526664734, -0.014454089105129242, 0.028958335518836975, -0.010045764967799187, 0.002268982119858265, -0.04019991308450699, 0.04625634476542473, -0.027185451239347458, -0.03542790934443474, -0.0420713797211647, -0.01675763726234436, -0.01896270364522934, 0.01892778091132641, 0.0601670928299427, -0.021555563434958458, -0.037875574082136154, 0.03040831722319126, -0.030295928940176964, 0.02151942439377308, -0.03385293111205101, 0.0009651

---

# Step 4: Create Vector Store with Chroma 🗄️

Store embeddings in Chroma for fast similarity search.Use OpenAI embeddings for OpenAI LLM. Use Hugingface embeddings for Hugging face model

In [12]:
from langchain_chroma import Chroma
import shutil

# Clean up any existing database
shutil.rmtree("./chroma_rag_db", ignore_errors=True)

# Create vector store
print("Creating vector store...")
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    #persist_directory="./chroma_rag_db",  # Persist to disk
    collection_name="rag_demo"
)

print(f"✓ Vector store created with {len(splits)} chunks")
print(f"✓ Persisted to: ./chroma_rag_db")

# Test retrieval
test_query = "What is RAG?"
test_results = vectorstore.similarity_search(test_query, k=2)

print(f"\nTest Query: '{test_query}'")
print(f"Top result: {test_results[0].page_content[:100]}...")

Creating vector store...
✓ Vector store created with 288 chunks
✓ Persisted to: ./chroma_rag_db

Test Query: 'What is RAG?'
Top result: needed — until it reaches the desired outcome and produces an Output/Action. It’s like 
having a pro...


**💡 What just happened:**
1. Each chunk was converted to a 384-dimensional vector
2. Vectors stored in Chroma with metadata
3. Chroma created an index for fast similarity search
4. Data persisted to disk (can reload later!)

---

# Step 5: Create a Retriever 🔍

Retriever finds the most relevant chunks for a query.

In [13]:
# Create retriever from vector store
retriever = vectorstore.as_retriever(
    search_type="similarity",  # Maximum Marginal Relevance (diverse results)
    search_kwargs={
        "k": 3,          # Return top 3 chunks
        #"fetch_k": 10    # Fetch 10 candidates for MMR selection
    }
)

print("✓ Retriever created with Similarity search")
print("  - Returns: 3 most relevant chunks")
#print("  - Strategy: MMR (diverse, non-redundant results)")

# Test retrieval
test_query = "What are React Agents?"
retrieved_docs = retriever.invoke(test_query)
#retrieved_docs = retriever.get_relevant_documents(test_query)

print(f"\nTest Query: '{test_query}'")
print(f"Retrieved {len(retrieved_docs)} chunks:\n")
for i, doc in enumerate(retrieved_docs, 1):
    print(f"{i}. {doc.page_content[:80]}...")
    print(f"   Source: {doc.metadata.get('source', 'N/A')}\n")

✓ Retriever created with Similarity search
  - Returns: 3 most relevant chunks

Test Query: 'What are React Agents?'
Retrieved 3 chunks:

1. ReAct – 
Reasoning Meets Action
Table 1.4: Characteristics of a fixed ReAct agen...
   Source: /content/Mastering AI Agents.pdf

2. 45
Mastering AI Agents
Define the ReAct Agent
Within your IDE of choice, you can...
   Source: /content/Mastering AI Agents.pdf

3. 14
Mastering AI Agents
Reasoning 
repeat until desired outcome achieved
Output /...
   Source: /content/Mastering AI Agents.pdf



**💡 Why MMR?**
- Standard search might return 3 very similar chunks
- MMR balances relevance + diversity
- Result: Better coverage, less redundancy

---

# Step 6: Set Up the LLM 🤖

Choose your LLM: OpenAI (paid) or HuggingFace (free).

## Option A: OpenAI (Best Quality)

In [14]:
# Uncomment if you have OpenAI API key
from langchain_openai import ChatOpenAI
# Ensure the API key is set as an environment variable if not already
if OPENAI_API_KEY and not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

model = ChatOpenAI(
    model="gpt-4o-mini",
   temperature=0  # 0 = focused, factual responses
)
print("✓ Using OpenAI GPT-4o-mini")

print("💡 OpenAI provides best quality but costs ~$0.002 per request")

✓ Using OpenAI GPT-4o-mini
💡 OpenAI provides best quality but costs ~$0.002 per request


##Option B : Google Gemini(Free)

In [ ]:
#google gemini
from langchain_google_genai import ChatGoogleGenerativeAI

# Ensure the API key is set as an environment variable if not already
if GEMINI_API_KEY and not os.getenv("GEMINI_API_KEY"):
    os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY

llm_gemini = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite")
print(llm_gemini.invoke("Hello, how are you?"))
print("✓ Using Google Gemini Pro")


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash-lite
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash-lite
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.0-flash-lite
Please retry in 55.653125342s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "

ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash-lite
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash-lite
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.0-flash-lite
Please retry in 53.510868762s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjec

## Option C: Using Groq Models

In [ ]:
from langchain_groq import ChatGroq

# Ensure the API key is set as an environment variable if not already
if GROQ_API_KEY and not os.getenv("GROQ_API_KEY"):
    os.environ["GROQ_API_KEY"] = GROQ_API_KEY

chat_groq = ChatGroq(model_name="qwen/qwen3-32b")
response = chat_groq.invoke("Hello, how are you?")
print(response.content)


<think>
Okay, the user said Hello, how are you? which is a greeting. I should respond with a friendly greeting as well. I need to acknowledge their greeting and offer assistance. Maybe add an emoji to keep it warm. I should keep it simple and open-ended to encourage them to ask their question or share what's on their mind. Let me check for any cultural nuances or tone issues. Yep, that sounds good. Ready to respond.
</think>

Hello! 😊 I'm here and ready to help. How can I assist you today?


**💡 Note:** HuggingFace free tier has rate limits. For production, use OpenAI or host your own model.

---

# Step 7: Build the RAG Chain 🔗

This is where the magic happens! Connect retriever + LLM.

In [22]:
from langchain_classic.chains import RetrievalQA
#from langchain.chains import create_retrieval_chain
from langchain_core.prompts import PromptTemplate

# Create a custom prompt
prompt_template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Do not hallucinate and answer only from the context provided.

Context:
{context}

Question: {question}

Answer: Let me help you with that."""

PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

# Create the RAG chain
qa_chain = RetrievalQA.from_chain_type(
    llm=model,
    chain_type="stuff",  # "stuff" = put all retrieved docs into prompt
    retriever=retriever,
    return_source_documents=True,  # Return sources with answer
    chain_type_kwargs={"prompt": PROMPT}
)

print("✓ RAG Chain created successfully!")
print("  - Retriever: Finds relevant chunks")
print("  - LLM: Generates answer from chunks")
print("  - Returns: Answer + source documents")

✓ RAG Chain created successfully!
  - Retriever: Finds relevant chunks
  - LLM: Generates answer from chunks
  - Returns: Answer + source documents


**💡 What's happening in the chain:**
1. User asks question
2. Retriever finds 3 most relevant chunks
3. Chunks inserted into prompt as "context"
4. LLM generates answer using that context
5. Sources returned for verification

---

# Step 8: Test Your RAG ChatBot! 🎉

Let's ask some questions!

In [23]:
def ask_question(question):
    """Ask a question and display answer with sources."""
    print(f"\n{'='*70}")
    print(f"Question: {question}")
    print(f"{'='*70}\n")

    result = qa_chain.invoke({"query": question})

    print(f"Answer:\n{result['result']}\n")
    print(f"{'='*70}")
    print(f"Sources:")
    for i, doc in enumerate(result['source_documents'], 1):
        print(f"\n{i}. {doc.metadata.get('source', 'Unknown')}")
        print(f"   {doc.page_content[:150]}...")
    print(f"{'='*70}")

# Test with sample questions
questions = [
    "What are the characteristics of React Agent?",
    "How does RAG work?",
    "What embedding models are commonly used?",
    "Why is document splitting important?"
]

# Ask the first question
ask_question(questions[1])


Question: How does RAG work?

Answer:
I don't know.

Sources:

1. /content/Mastering AI Agents.pdf
   needed — until it reaches the desired outcome and produces an Output/Action. It’s like 
having a problem solver who not only thinks and acts but also ...

2. /content/Mastering AI Agents.pdf
   Examples
Legal research tools, medical assistants referencing clinical studies, technical 
troubleshooting agents.
ReAct + RAG Agent
Input Query
Knowl...

3. /content/Mastering AI Agents.pdf
   accomplish end-to-end tasks?” This singular question opens up a door: AI agents. A RAG 
system helps an LLM provide accurate answers based on given co...


In [ ]:
# Try more questions
ask_question(questions[1])

In [20]:
# Ask your own question!
custom_question = "What is Chroma and why use it?"  # Change this!
ask_question(custom_question)


Question: What is Chroma and why use it?

Answer:
I don't know.

Sources:

1. /content/Mastering AI Agents.pdf
   LangChain’s tools. It allows you to define and integrate custom tools tailored to your 
specific needs. This capability is ideal if you’re looking to ...

2. /content/Mastering AI Agents.pdf
   valuable.
Moving on to Autogen, this framework stands out with its support for various tools, 
including code executors and function callers. Its modu...

3. /content/Mastering AI Agents.pdf
   group chats. It lets you easily define complex interaction patterns between agents. Its 
graph-based approach aids in visualizing and managing these i...


In [ ]:
# Ask your own question!
custom_question = "Who is Donald Trump?"  # Change this!
ask_question(custom_question)


Question: Who is Donald Trump?

Answer:
Let me help you with that. Based on the provided context, I don't have information about who Donald Trump is. The context discusses Chroma, Retrieval Augmented Generation (RAG), and vector embeddings.

Sources:

1. chroma_db.txt
   Chroma is an open-source embedding database designed for AI applications. 
        It provides fast similarity search, easy integration with LangChain...

2. rag_explanation.txt
   Retrieval Augmented Generation (RAG) is a technique that combines information retrieval 
        with text generation. It works by first retrieving re...

3. embeddings_guide.txt
   Vector embeddings are numerical representations of text that capture semantic meaning. 
        Similar texts produce similar embedding vectors. Commo...


**💡 Notice:**
- ✅ Answers are grounded in your documents
- ✅ Sources are provided for verification
- ✅ No hallucination (answers "I don't know" when info not in docs)

**This is the power of RAG!**

---

# Step 9: Interactive Chat Interface 💬

Let's create a simple chat loop for continuous interaction.

In [24]:
def chat_loop():
    """
    Simple chat interface.
    Type 'quit' or 'exit' to end.
    """
    print("\n" + "="*70)
    print("RAG ChatBot Ready! Ask me anything about the documents.")
    print("Type 'quit' or 'exit' to end the conversation.")
    print("="*70 + "\n")

    while True:
        # Get user input
        question = input("\nYou: ").strip()

        # Check for exit
        if question.lower() in ['quit', 'exit', 'q']:
            print("\nGoodbye! 👋")
            break

        if not question:
            continue

        # Get answer
        try:
            result = qa_chain.invoke({"query": question})
            print(f"\nBot: {result['result']}")
            print(f"\n📚 Sources: {', '.join([doc.metadata.get('source', 'N/A') for doc in result['source_documents']])}")
        except Exception as e:
            print(f"\nError: {e}")
            print("Please try again.")

# Uncomment to start chat
chat_loop()

print("💡 Uncomment 'chat_loop()' above to start interactive chat!")


RAG ChatBot Ready! Ask me anything about the documents.
Type 'quit' or 'exit' to end the conversation.


You: What is the capital of India

Bot: I don't know.

📚 Sources: /content/Mastering AI Agents.pdf, /content/Mastering AI Agents.pdf, /content/Mastering AI Agents.pdf

You: Who is the prime minister of India

Bot: I don't know.

📚 Sources: /content/Mastering AI Agents.pdf, /content/Mastering AI Agents.pdf, /content/Mastering AI Agents.pdf

You: Give me an example of vector database

Bot: I don't know.

📚 Sources: /content/Mastering AI Agents.pdf, /content/Mastering AI Agents.pdf, /content/Mastering AI Agents.pdf

You: What are the types of AI Agents

Bot: The context provided mentions "Fixed Automation: The Digital Assembly Line" as one type of AI agent, characterized by no intelligence, predictable behavior, and limited scope. However, it does not provide a comprehensive list of all types of AI agents. Therefore, I don't know the complete answer regarding all the types of AI agent

---

# Bonus: Simple Streamlit UI 🎨

Want a web interface? Create `app.py` with this code:

In [ ]:
streamlit_code = '''
import streamlit as st
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFaceEndpoint
from langchain.chains import RetrievalQA

# Page config
st.set_page_config(page_title="RAG ChatBot", page_icon="🤖")
st.title("🤖 RAG ChatBot")
st.caption("Ask questions about your documents!")

# Load components (cached)
@st.cache_resource
def load_rag_chain():
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    vectorstore = Chroma(
        persist_directory="./chroma_rag_db",
        embedding_function=embeddings,
        collection_name="rag_demo"
    )
    retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 3})
    llm = HuggingFaceEndpoint(repo_id="mistralai/Mistral-7B-Instruct-v0.2", temperature=0.1)

    return RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        return_source_documents=True
    )

qa_chain = load_rag_chain()

# Chat interface
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat history
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.write(message["content"])

# User input
if prompt := st.chat_input("Ask a question..."):
    # Add user message
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user"):
        st.write(prompt)

    # Get response
    with st.chat_message("assistant"):
        with st.spinner("Thinking..."):
            result = qa_chain.invoke({"query": prompt})
            response = result["result"]
            sources = [doc.metadata.get("source", "N/A") for doc in result["source_documents"]]

            st.write(response)
            st.caption(f"📚 Sources: {', '.join(set(sources))}")

    # Add assistant message
    st.session_state.messages.append({"role": "assistant", "content": response})
'''

# Save to file
with open('streamlit_app.py', 'w') as f:
    f.write(streamlit_code)

print("✓ Created streamlit_app.py")
print("\nTo run:")
print("  pip install streamlit")
print("  streamlit run streamlit_app.py")

---

# What You've Built! 🎉

**Congratulations!** You now have a fully functional RAG chatbot that:

✅ **Loads documents** from multiple sources (PDF, web, text)
✅ **Splits intelligently** using RecursiveCharacterTextSplitter
✅ **Creates embeddings** using HuggingFace (free!)
✅ **Stores in Chroma** with persistent storage
✅ **Retrieves with MMR** for diverse, relevant results
✅ **Generates answers** using LLM (Mistral or GPT)
✅ **Shows sources** for verification
✅ **Has a UI** (Streamlit)

**All in ~30 minutes!**

---

## Performance & Cost

**With Free Options (HuggingFace):**
- 💰 Cost: $0
- ⚡ Speed: 2-5 seconds per query
- 📊 Quality: Good for most use cases

**With OpenAI:**
- 💰 Cost: ~$0.002 per query
- ⚡ Speed: <1 second per query
- 📊 Quality: Excellent

---

## Next Steps 🚀

**You can now:**
1. Replace sample docs with YOUR documents
2. Adjust chunk size/overlap for your use case
3. Try different embedding models
4. Experiment with retrieval parameters
5. Customize the prompt for your domain

**In Part 5b (Coming Next):**
- 🔥 Conversational RAG (chat history)
- 🔥 Advanced retrieval (re-ranking, hybrid search)
- 🔥 Evaluation metrics (measure quality)
- 🔥 Production optimization (caching, streaming)
- 🔥 Error handling & edge cases

---

## Common Issues & Solutions

**Problem: "Rate limit exceeded" with HuggingFace**
- Solution: Use OpenAI or host model locally with Ollama

**Problem: "Slow responses"**
- Solution: Use GPU for embeddings, try smaller chunks

**Problem: "Answers not accurate"**
- Solution: Adjust chunk size, try k=5 instead of k=3, improve prompts

**Problem: "Can't find answer in documents"**
- Solution: Check if info is actually in docs, try similarity search directly

---

## Resources

- [LangChain Documentation](https://python.langchain.com/)
- [Chroma Documentation](https://docs.trychroma.com/)
- [HuggingFace Models](https://huggingface.co/models)
- [My Previous Blogs](#) - Parts 1-4 of this series

---

# Thank You! 🙏

**You've completed the journey:**
1. ✅ Document Loading
2. ✅ Document Splitting
3. ✅ Vector Embeddings
4. ✅ Vector Databases
5. ✅ **Building RAG ChatBot** ← You are here!

**What's next?** Try it with YOUR documents and share your results!

**Questions?** Drop them in the comments below!

**Stay tuned for Part 5b** - Production RAG patterns! 🚀

#LangChain #RAG #AI #MachineLearning #Python #ChatBot #VectorDatabase #LLM